In [2]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'langchain_api_key'

In [4]:
os.environ['OPENAI_API_KEY'] = 'model_api_key' 

In [5]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [6]:
# Load Documents
pdf_path = "/home/qasob/knowledge_model_composition/data/Book_Introduction to Robotics _JJ_Craig.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [7]:
template_expand = """You are an AI language model assistant. Your task is to generate three 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}
"""


prompt_expand = ChatPromptTemplate.from_template(template_expand)

In [8]:
template_robotics = """You are a robotics tutor helping students understand concepts from robotics textbooks.  
Use ONLY the provided context to answer the question.  
If the context does not contain the answer, say: "The context does not provide enough information."

### Context:
{context}

### Question:
{question}

### Instructions:
- Base your answer ONLY on the given context (do not use outside knowledge).
- Provide a clear, concise explanation suitable for a student.
- If relevant, reference the part of the context you used (e.g., section or formula).
- Avoid adding information not supported by the context.

### Answer:
"""

prompt_robotics = ChatPromptTemplate.from_template(template_robotics)

In [9]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [10]:
def rag_pipeline(user_question):
    expanded = (prompt_expand | llm).invoke({"question": user_question}).content
    queries = [q.strip() for q in expanded.split("\n") if q.strip()]

    answers = []

    # For each expanded query → retrieve context → answer
    for q in queries:
        docs = retriever.get_relevant_documents(q)
        context = "\n\n".join([d.page_content for d in docs])
        response = (prompt_robotics | llm).invoke({"context": context, "question": q}).content
        answers.append(response)

    # Combine answers by uniqueness 
    merge_prompt = ChatPromptTemplate.from_template("""
Combine the following answers into a single coherent explanation. 
Remove duplicates, keep unique insights, and ensure the result is clear and comprehensive.

Answers:
{answers}

Final Answer:
""")

    final_answer = (merge_prompt | llm).invoke({"answers": "\n\n".join(answers)}).content
    return final_answer

In [12]:
result = rag_pipeline("What is kinematic?")
print(result)

The term "kinematic" in physics refers to the science of motion, focusing on the study of position, velocity, acceleration, and higher order derivatives of motion without considering the forces that cause it. In robotics, particularly in the study of manipulators, kinematics emphasizes the geometrical and time-based properties of motion. This includes understanding how manipulators move and position themselves in space, which is crucial for tasks such as static positioning.

Kinematics is foundational for designing and controlling robotic systems, as it defines the relationships between different motions. It is essential to grasp kinematics before delving into dynamics, which involves the forces and torques that cause motion. Key principles of kinematic motion include the analysis of position, velocity, acceleration, and the relationships among these variables over time.

In the context of manipulators, kinematics involves understanding how nearly rigid links are connected by joints th